# AB Multi-arm Star Polymers in Potential Fields

This tutorial demonstrates how to compute partition functions, propagators, and concentrations of **AB multi-arm star polymers** in external potential fields.

**What you'll learn:**
- Setting up multi-arm star polymer architectures
- Computing partition functions and concentrations for star polymers

**Prerequisites:** Complete `00_QuickStart.ipynb` and `Diblock.ipynb` first.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["MKL_NUM_THREADS"] = "1"      # Single-threaded MKL

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from polymerfts import PropagatorSolver
import polymerfts

## 2. Simulation Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| $N$ | 100 | Reference chain length (number of statistical segments) |
| $\Delta s$ | 0.01 | Contour step size (= $1/N$) |
| $L_x, L_y$ | $5.0 R_0$ | Box dimensions in units of $R_0 = b\sqrt{N}$ |
| $m_x, m_y$ | 64 | Grid points per dimension |
| $b_A/b, b_B/b$ | 1.0 | Statistical segment lengths (relative to reference $b$) |
| $f_A$ | 0.7 | A block fraction per arm |
| $f_B$ | 0.3 | B block fraction per arm |
| Arms | 3 | Number of arms in the star polymer |

In [ ]:
# Simulation parameters
nx = [64, 64]              # Grid points
lx = [5.0, 5.0]            # Box size (in units of R0 = b*sqrt(N))
ds = 0.01                  # Contour step interval

# Block fractions per arm
f_A = 0.7
f_B = 0.3

## 3. Create the Solver

We use `PropagatorSolver` for a clean interface. For star polymers, we define the architecture using graph notation where multiple arms connect at a central node.

Graph notation: Node 0 is the center, arms extend to nodes 1,2,3 then to 4,5,6
```
       B(4)           B(5)           B(6)
        |              |              |
       A(1)           A(2)           A(3)
         \             |             /
          \            |            /
           -------- (0) center ----
```

In [ ]:
# Create solver
solver = PropagatorSolver(
    nx=nx,
    lx=lx,
    ds=ds,
    bond_lengths={"A": 1.0, "B": 1.0},
    bc=["periodic", "periodic", "periodic", "periodic"],
    chain_model="continuous",
    method="pseudospectral",
    platform="cpu-mkl",
    reduce_memory=False
)

# Add AB 3-arm star polymer where each arm is a diblock with f_A=0.7 and f_B=0.3
solver.add_polymer(
    volume_fraction=1.0,
    blocks=[
        ["A", f_A, 0, 1], ["B", f_B, 1, 4],  # Arm 1
        ["A", f_A, 0, 2], ["B", f_B, 2, 5],  # Arm 2
        ["A", f_A, 0, 3], ["B", f_B, 3, 6],  # Arm 3
    ]
)

print(solver.info)

## 4. Set Potential Fields

We create sinusoidal potential fields that are opposite for A and B monomers.

In [ ]:
# Create sinusoidal potential fields (opposite for A and B)
w_A = np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))
w_B = -w_A  # Opposite preference

# Visualize the potential fields
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Potential Fields", fontsize=14)

im0 = axes[0].imshow(w_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
im1 = axes[1].imshow(w_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
axes[0].set(title='$w_A$ (A monomers)', xlabel='y', ylabel='x')
axes[1].set(title='$w_B$ (B monomers)', xlabel='y', ylabel='x')

plt.colorbar(im0, ax=axes[0])
plt.colorbar(im1, ax=axes[1])
plt.tight_layout()
plt.show()

## 5. Compute Partition Function

The propagators for A-type blocks, $(v,u) \in \{(0,1),(0,2),(0,3),(1,0),(2,0),(3,0)\}$:

$$\frac{\partial q^{v \rightarrow u}_0(\mathbf{r}, s)}{\partial s} = \frac{b_A^2}{6}\nabla^2 q^{v \rightarrow u}_0(\mathbf{r}, s) - w_A(\mathbf{r}) q^{v \rightarrow u}_0(\mathbf{r}, s), \quad s\in [0, f_A]$$

The propagators for B-type blocks, $(v,u) \in \{(1,4),(2,5),(3,6),(4,1),(5,2),(6,3)\}$:

$$\frac{\partial q^{v \rightarrow u}_0(\mathbf{r}, s)}{\partial s} = \frac{b_B^2}{6}\nabla^2 q^{v \rightarrow u}_0(\mathbf{r}, s) - w_B(\mathbf{r}) q^{v \rightarrow u}_0(\mathbf{r}, s), \quad s\in [0, f_B]$$

Single chain partition function:

$$Q_0 = \frac{1}{V}\int d\mathbf{r}\, q^{0 \rightarrow 1}_0(\mathbf{r}, s) \cdot q^{1 \rightarrow 0}_0(\mathbf{r}, f_A-s)$$

In [ ]:
# Compute propagators
solver.compute_propagators({"A": w_A.flatten(), "B": w_B.flatten()})

# Get single chain partition function
Q = solver.get_partition_function(polymer=0)
print(f"Single chain partition function Q = {Q:.6f}")
print(f"\nInterpretation: Q > 1 means the potential field favors certain conformations")

## 6. Compute Concentrations

$$\phi_A(\mathbf{r}) = \frac{1}{Q_0} \sum_{(v,u)\in\{(0,1),(0,2),(0,3)\}} \int_{0}^{f_A} ds\, q^{v \rightarrow u}_0(\mathbf{r}, s) \cdot q^{u \rightarrow v}_0(\mathbf{r}, f_A-s)$$

$$\phi_B(\mathbf{r}) = \frac{1}{Q_0} \sum_{(v,u)\in\{(1,4),(2,5),(3,6)\}} \int_{0}^{f_B} ds\, q^{v \rightarrow u}_0(\mathbf{r}, s) \cdot q^{u \rightarrow v}_0(\mathbf{r}, f_B-s)$$

In [ ]:
# Compute concentrations
solver.compute_concentrations()

# Get ensemble-averaged concentrations
phi_A = np.reshape(solver.get_concentration("A"), nx)
phi_B = np.reshape(solver.get_concentration("B"), nx)

# Visualize concentrations
vmin = min(phi_A.min(), phi_B.min())
vmax = max(phi_A.max(), phi_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Monomer Concentrations", fontsize=14)

im0 = axes[0].imshow(phi_A, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0].set(title='$\phi_A$ (A concentration)', xlabel='y', ylabel='x')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(phi_B, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[1].set(title='$\phi_B$ (B concentration)', xlabel='y', ylabel='x')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

print("\nObservation: A monomers concentrate where w_A < 0")
print("             B monomers concentrate where w_B < 0")

## 7. Summary

In this tutorial, we learned:

1. **Star polymer architecture**: Multiple arms connect at a central node (node 0)
2. **Graph notation**: Each arm is defined by blocks connecting the center to outer nodes
3. **PropagatorSolver** handles the setup automatically

### Next Steps

- **Comb polymers**: See `BranchedComb.ipynb` for backbone with side chains
- **Polymer mixtures**: See `Mixture.ipynb`
- **Accessing individual propagators**: Set `reduce_memory=False` to store all propagator steps (see `Diblock.ipynb` for examples)